# UNIQLO Deal Research
Deep analysis of deal patterns, timing, seasonality, and price predictions.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from analysis.queries import (
    get_engine, load_timeseries, load_price_history,
    load_deal_heatmap, load_seasonal, load_top_products
)
from analysis.features import engineer_features
from analysis.predictions import predict_price, deal_probability, price_drop_probability, best_time_to_buy

engine = get_engine()
print('Connected to Supabase')

## 1. Load Data

In [ ]:
# --- CONFIG ---
FILTER_SIZE   = 'M'       # set to None to see all sizes
FILTER_GENDER = None      # 'woman', 'man', or None
DAYS_BACK     = None      # last N days, or None for all history

df_raw = load_timeseries(engine, size=FILTER_SIZE, gender=FILTER_GENDER, days=DAYS_BACK)
df = engineer_features(df_raw)

print(f'Loaded {len(df):,} rows')
print(f'Date range: {df["fetched_at"].min().date()} → {df["fetched_at"].max().date()}')
print(f'Unique products: {df["product_id"].nunique()}')
print(f'Sizes: {sorted(df["size"].unique())}')
df.head(3)

## 2. When Do Best Deals Appear? (Day × Hour Heatmap)

In [ ]:
heatmap_df = load_deal_heatmap(engine, size=FILTER_SIZE)

pivot = heatmap_df.pivot(index='hour', columns='day_of_week', values='deal_rate').fillna(0)
pivot.columns = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']

fig = go.Figure(go.Heatmap(
    z=pivot.values,
    x=pivot.columns.tolist(),
    y=[f'{h:02d}:00' for h in pivot.index],
    colorscale='RdYlGn',
    text=np.round(pivot.values * 100, 1),
    texttemplate='%{text}%',
    hovertemplate='%{x} %{y}<br>Deal rate: %{z:.1%}<extra></extra>'
))
fig.update_layout(
    title=f'Deal Probability by Day & Hour (size={FILTER_SIZE or "all"})',
    xaxis_title='Day of Week',
    yaxis_title='Hour (UTC)',
    height=600
)
fig.show()

# Best time
best = best_time_to_buy(pivot)
print(f"\nBest time to check: {best['best_day']} at {best['best_hour']:02d}:00 → {best['probability']*100:.1f}% deal probability")

## 3. Deal Rate by Day of Week

In [ ]:
day_stats = (
    df.groupby('day_name')
    .agg(total=('id','count'), good_deals=('is_good_deal','sum'), avg_discount=('discount_percent','mean'))
    .reset_index()
)
day_stats['deal_rate'] = day_stats['good_deals'] / day_stats['total']
day_order = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
day_stats['day_name'] = pd.Categorical(day_stats['day_name'], categories=day_order, ordered=True)
day_stats = day_stats.sort_values('day_name')

fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_bar(x=day_stats['day_name'], y=day_stats['deal_rate'], name='Deal Rate', marker_color='#2ecc71')
fig.add_scatter(x=day_stats['day_name'], y=day_stats['avg_discount'],
                name='Avg Discount %', mode='lines+markers', marker_color='#e74c3c', secondary_y=True)
fig.update_layout(title='Deal Rate & Average Discount by Day of Week', height=400)
fig.update_yaxes(title_text='Deal Rate', tickformat='.0%', secondary_y=False)
fig.update_yaxes(title_text='Avg Discount %', secondary_y=True)
fig.show()

## 4. Size Analysis — Which Sizes Go On Sale Most

In [ ]:
df_all_sizes = engineer_features(load_timeseries(engine))

size_stats = (
    df_all_sizes.groupby('size')
    .agg(
        observations=('id','count'),
        good_deals=('is_good_deal','sum'),
        avg_discount=('discount_percent','mean'),
        min_price=('promo_price','min'),
        avg_price=('promo_price','mean')
    ).reset_index()
)
size_stats['deal_rate'] = size_stats['good_deals'] / size_stats['observations']
size_stats = size_stats.sort_values('deal_rate', ascending=False)

fig = px.bar(
    size_stats,
    x='size', y='deal_rate',
    color='avg_discount',
    color_continuous_scale='RdYlGn',
    text=size_stats['deal_rate'].map(lambda x: f'{x:.0%}'),
    hover_data=['observations','good_deals','avg_discount','min_price'],
    title='Deal Rate by Size (color = avg discount)'
)
fig.update_layout(height=450, xaxis_title='Size', yaxis_title='Deal Rate', yaxis_tickformat='.0%')
fig.show()

## 5. Seasonal Component

In [ ]:
seasonal_df = load_seasonal(engine)

# Aggregate across sizes for overview
monthly = (
    seasonal_df.groupby(['year','month'])
    .agg(avg_discount=('avg_discount','mean'), good_deals=('good_deals','sum'),
         observations=('observations','sum'), min_price=('min_price','min'))
    .reset_index()
)
monthly['deal_rate'] = monthly['good_deals'] / monthly['observations']
monthly['date'] = pd.to_datetime(monthly[['year','month']].assign(day=1))
monthly = monthly.sort_values('date')

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=['Avg Discount % Over Time', 'Deal Rate Over Time'])

fig.add_scatter(x=monthly['date'], y=monthly['avg_discount'],
                mode='lines+markers', name='Avg Discount', marker_color='#e74c3c', row=1, col=1)
fig.add_scatter(x=monthly['date'], y=monthly['deal_rate'],
                mode='lines+markers', name='Deal Rate', marker_color='#2ecc71',
                fill='tozeroy', row=2, col=1)

fig.update_layout(title='Seasonal Deal Patterns', height=600)
fig.update_yaxes(tickformat='.0%', row=2)
fig.show()

# By season
season_map = {12:'Winter',1:'Winter',2:'Winter',3:'Spring',4:'Spring',5:'Spring',
              6:'Summer',7:'Summer',8:'Summer',9:'Autumn',10:'Autumn',11:'Autumn'}
monthly['season'] = monthly['month'].map(season_map)
season_agg = monthly.groupby('season').agg(avg_discount=('avg_discount','mean'), deal_rate=('deal_rate','mean')).reset_index()

fig2 = px.bar(season_agg, x='season', y='deal_rate', color='avg_discount',
              color_continuous_scale='RdYlGn', text=season_agg['deal_rate'].map(lambda x: f'{x:.0%}'),
              title='Deal Rate by Season', category_orders={'season':['Winter','Spring','Summer','Autumn']})
fig2.update_layout(height=400, yaxis_tickformat='.0%')
fig2.show()

## 6. Top Products — Most Frequent Deals

In [ ]:
top = load_top_products(engine)
top['deal_rate'] = top['good_deal_count'] / top['observations']

fig = px.scatter(
    top.head(30),
    x='avg_discount', y='deal_rate',
    size='max_reviews', color='avg_rating',
    color_continuous_scale='RdYlGn',
    hover_name='name',
    hover_data=['min_price_ever','max_discount','gender'],
    text='name',
    title='Top Products: Discount vs Deal Frequency (size = reviews, color = rating)'
)
fig.update_traces(textposition='top center', textfont_size=9)
fig.update_layout(height=600, xaxis_title='Avg Discount %', yaxis_title='Deal Rate', yaxis_tickformat='.0%')
fig.show()

## 7. Price Prediction for a Specific Product

In [ ]:
# Pick a product from top list or set manually
PRODUCT_ID    = top.iloc[0]['product_id']   # change to any product_id
HORIZON_DAYS  = 30
TARGET_PRICE  = None   # set a price to calculate drop probability, e.g. 15.0

ph = load_price_history(engine, PRODUCT_ID)
print(f"Product: {ph['name'].iloc[0]}")
print(f"Observations: {len(ph)} | Price range: {ph['promo_price'].min()}€ → {ph['promo_price'].max()}€")

forecast = predict_price(ph, horizon_days=HORIZON_DAYS)

if forecast:
    fig = go.Figure()

    # Historical
    fig.add_scatter(x=ph['fetched_at'], y=ph['promo_price'],
                    mode='markers+lines', name='Historical Price',
                    marker=dict(color=ph['action'].map({
                        'SUPER':'#27ae60','GOOD DEAL':'#2ecc71','BIG DISCOUNT':'#f39c12',
                        'VERY CHEAP':'#f1c40f','NEUTRAL':'#95a5a6','AVOID':'#e74c3c'
                    }).fillna('#95a5a6'), size=8), line_color='#3498db')

    # Forecast
    fig.add_scatter(x=forecast['forecast_dates'], y=forecast['forecast_prices'],
                    mode='lines', name='Predicted Price', line=dict(color='#e67e22', dash='dash'))

    # Confidence interval
    fig.add_scatter(
        x=list(forecast['forecast_dates']) + list(forecast['forecast_dates'])[::-1],
        y=list(forecast['ci_upper']) + list(forecast['ci_lower'])[::-1],
        fill='toself', fillcolor='rgba(230,126,34,0.15)',
        line_color='rgba(0,0,0,0)', name='80% Confidence Interval'
    )

    # Min ever line
    fig.add_hline(y=forecast['min_price_ever'], line_dash='dot', line_color='green',
                  annotation_text=f"Min ever: {forecast['min_price_ever']}€")

    if TARGET_PRICE:
        fig.add_hline(y=TARGET_PRICE, line_dash='dot', line_color='red',
                      annotation_text=f"Target: {TARGET_PRICE}€")

    fig.update_layout(
        title=f"{ph['name'].iloc[0]} — Price Trend & {HORIZON_DAYS}d Forecast (trend: {forecast['trend']})",
        xaxis_title='Date', yaxis_title='Promo Price (€)', height=500,
        legend=dict(orientation='h', yanchor='bottom', y=1.02)
    )
    fig.show()

    print(f"\nCurrent price:          {forecast['current_price']}€")
    print(f"Trend:                  {forecast['trend']}")
    print(f"Predicted min in {HORIZON_DAYS}d:   {forecast['expected_min_in_horizon']:.2f}€")
    print(f"Historical min:         {forecast['min_price_ever']}€")

    if TARGET_PRICE:
        drop = price_drop_probability(ph, TARGET_PRICE)
        print(f"\nP(price ≤ {TARGET_PRICE}€):      {drop['probability']*100:.1f}%")
        print(f"Historical % below {TARGET_PRICE}€: {drop['pct_below_target']*100:.1f}%")

## 8. Deal Probability Map for Current Filters

In [ ]:
prob_pivot = deal_probability(df)

fig = go.Figure(go.Heatmap(
    z=prob_pivot.values,
    x=prob_pivot.columns.tolist(),
    y=[f'{h:02d}:00' for h in prob_pivot.index],
    colorscale='RdYlGn',
    zmin=0, zmax=1,
    text=np.round(prob_pivot.values * 100, 1),
    texttemplate='%{text}%',
    hovertemplate='%{x} %{y}<br>P(good deal) = %{z:.1%}<extra></extra>'
))
fig.update_layout(
    title=f'Deal Probability Map — Size: {FILTER_SIZE or "all"}, Gender: {FILTER_GENDER or "all"}',
    xaxis_title='Day of Week', yaxis_title='Hour (UTC)', height=600
)
fig.show()

best = best_time_to_buy(prob_pivot)
print(f"Best time: {best['best_day']} at {best['best_hour']:02d}:00 → {best['probability']*100:.1f}% probability")

## 9. Price Distribution by Action

In [ ]:
action_colors = {
    'SUPER': '#27ae60', 'GOOD DEAL': '#2ecc71', 'BIG DISCOUNT': '#f39c12',
    'VERY CHEAP': '#f1c40f', 'CHEAP UPPER MID': '#1abc9c',
    'NEUTRAL': '#95a5a6', 'AVOID': '#e74c3c', 'DECENT': '#3498db'
}

fig = go.Figure()
for action in df['action'].unique():
    subset = df[df['action'] == action]['promo_price'].dropna()
    if len(subset) > 5:
        fig.add_violin(y=subset, name=action,
                       box_visible=True, meanline_visible=True,
                       fillcolor=action_colors.get(action, '#aaa'),
                       opacity=0.7, line_color='white')

fig.update_layout(
    title='Price Distribution by Deal Action',
    yaxis_title='Promo Price (€)', height=500,
    violinmode='group'
)
fig.show()

## 10. Summary Stats

In [ ]:
good = df[df['is_good_deal']]

print('=== SUMMARY ===')
print(f"Total observations:      {len(df):,}")
print(f"Good deals:              {len(good):,} ({len(good)/len(df)*100:.1f}%)")
print(f"Unique products:         {df['product_id'].nunique()}")
print(f"Avg discount (all):      {df['discount_percent'].mean():.1f}%")
print(f"Avg discount (deals):    {good['discount_percent'].mean():.1f}%")
print(f"Cheapest price seen:     {df['promo_price'].min():.2f}€")
print(f"\nTop actions:")
print(df['action'].value_counts().to_string())
print(f"\nTop seasons for deals:")
print(good.groupby('season')['is_good_deal'].count().sort_values(ascending=False).to_string())